<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/data-designer/healthcare-datasets/physician-notes-with-realistic-personal-details.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🧑‍⚕️ Generating Realistic Patient Data & Physician Notes

This notebook demonstrates how to use Gretel's Data Designer SDK to generate realistic patient data including physician notes. We'll leverage both structured data generation and LLM capabilities to create a comprehensive medical dataset.

## 🛠️ Setup and Installation

First, let's install the necessary packages:

In [ ]:
%%capture
%pip install -U gretel_client datasets

## 🔑 Initialize Gretel Client

We import the necessary libraries and initialize the Gretel client. The API key is set to "prompt" which will prompt you for your API key if needed.

In [ ]:
from datasets import load_dataset

from gretel_client.navigator_client import Gretel

gretel = Gretel(api_key="prompt")

## 📊 Loading Seed Data

We'll use Gretel's symptom-to-diagnosis dataset as our seed data. This dataset contains patient symptoms and corresponding diagnoses which will help generate realistic medical scenarios.

In [ ]:
# Let's use Gretel's symptom-to-diagnosis dataset to seed our workflow.
df_seed = load_dataset("gretelai/symptom_to_diagnosis")["train"].to_pandas()
df_seed = df_seed.rename(columns={"output_text": "diagnosis", "input_text": "patient_summary"})

print(f"Number of records: {len(df_seed)}")

df_seed.head()

## 🧩 Data Designer Configuration

Let's create a new Data Designer instance and configure it with our seed dataset and person samplers. We'll use person samplers to generate realistic patient and doctor information.

In [ ]:
aidd = gretel.data_designer.new(model_suite="apache-2.0")

# We use with_replacement=False, so our max num_records is 853.
aidd.with_seed_dataset(
    df_seed,
    sampling_strategy="shuffle", # "ordered"
    with_replacement=True
)

# Create a couple random person samplers. For now, the
# default locale has been updated to "en_GB", since we
# do not yet support the PGM in streaming mode.
aidd.with_person_samplers({"patient_sampler": {}, "doctor_sampler": {}})

## 🏗️ Defining Data Structure

Now we'll define the structure of our dataset by adding columns for patient information, dates, and medical details. We'll use:

- `uuid` for patient identification
- Patient personal information (`first_name`, `last_name`, `dob`, `patient_email`)
- Medical timeline information (`symptom_onset_date`, `date_of_visit`)
- Physician information (`physician`)

In [ ]:
aidd.add_column(
    name="patient_id",
    type="uuid",
    params={"prefix": "PT-", "short_form": True, "uppercase": True},
)

aidd.add_column(
    name="first_name",
    type="expression",
    expr="{{patient_sampler.first_name}}"
)

aidd.add_column(
    name="last_name",
    type="expression",
    expr="{{patient_sampler.last_name}}"
)


aidd.add_column(
    name="dob",
    type="expression",
    expr="{{patient_sampler.birth_date}}"
)


aidd.add_column(
    name="patient_email",
    type="expression",
    expr="{{patient_sampler.email_address}}"
)


aidd.add_column(
    name="symptom_onset_date",
    type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

aidd.add_column(
    name="date_of_visit",
    type="timedelta",
    params={
        "dt_min": 1,
        "dt_max": 30,
        "reference_column_name": "symptom_onset_date"
    },
)

aidd.add_column(
    name="physician",
    type="expression",
    expr="Dr. {{doctor_sampler.first_name}} {{doctor_sampler.last_name}}",
)

### 📝 LLM-Generated Physician Notes

The final and most complex column uses an LLM to generate realistic physician notes. We provide:

- Context about the patient and their condition
- Patient summary from our seed data
- Clear formatting instructions

This will create detailed medical notes that reflect the patient's diagnosis and visit information. Note how we reference other columns in the prompt using Jinja templating syntax with double curly braces `{{column_name}}`.

In [ ]:
# Note we have access to the seed data fields.
aidd.add_column(
    name="physician_notes",
    type="llm-text",
    prompt="""\
<context>
You are a primary-care physician who just had an appointment with {{first_name}} {{last_name}},
who has been struggling with symptoms from {{diagnosis}} since {{symptom_onset_date}}.
The date of today's visit is {{date_of_visit}}.
</context>

<patient_summary_of_symptoms>
{{patient_summary}}
</patient_summary_of_symptoms>

<task>
Write careful notes about your visit with {{first_name}},
as {{physician}}.

Format the notes as a busy doctor might.
</task>
"""
 )

## 👁️ Previewing the Dataset

Let's generate a preview to see how our data looks before creating the full dataset. This helps verify that our configuration is working as expected.

In [ ]:
preview = aidd.preview(verbose_logging=True)

### 🔍 Examining a Sample Record

Let's take a closer look at a single record to inspect the details of our generated data:

In [ ]:
preview.display_sample_record()

### 📊 Examining the Complete Dataset

Now let's look at the full preview dataset as a DataFrame to see all generated columns:

In [ ]:
# The full dataset includes the seed data as columns.
preview.dataset.df

## 🚀 Generating the Full Dataset

Now that we've verified our configuration works correctly, let's generate a larger dataset with 100 records. We'll wait for the workflow to complete so we can access the data immediately.

In [ ]:
workflow_run = aidd.create(
    num_records=100,
    name="physician_notes"
)
workflow_run.wait_until_done()